# RD sim on spheres
### (uniform parameter values)

In [ ]:
%load_ext autoreload
%autoreload 1

import os
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from joblib import Parallel, delayed

from rdmb import *
from quant import *
%aimport rdmb, quant

# %matplotlib inline
%matplotlib notebook

In [ ]:
def run_rdmb_uni_AC(fnbase, out_dir, AC):
    
    mb_obj_file = "mbobj/mb_1.obj.gz"
    A = AC[0]
    C = AC[1]
    out_file_base = out_dir + "/" + fnbase + "_A{:.4f}_C{:.3f}".format(A, C)
    
    rdmb.rdmb_uni_AC(mb_obj_file, out_file_base, A=A, C=C, max_ite=2000)
    
    return out_file_base

### RD calculation (parallelized)

In [ ]:
%%time

fnbase = "MB_1"
out_dir = "rd_out/uni"

try:
    os.makedirs("./"+out_dir)
except FileExistsError:
    pass

As = np.linspace(0.07, 0.14, 30)
Cs = np.linspace(-0.1, 0.4, 30)

AAs, CCs = np.meshgrid(As, Cs)

np.random.seed(42)

rdmb_fnbases = Parallel(n_jobs=36, verbose=1, max_nbytes=None)(
                    [delayed(run_rdmb_uni_AC)(
                        fnbase=fnbase,
                        out_dir=out_dir,
                        AC=AC)
                     for AC in zip(AAs.flatten(), CCs.flatten())])


### Rendering for quantificaiton

In [ ]:
pngfnames_q = Parallel(n_jobs=36, verbose=1, max_nbytes=None)(
                    [delayed(rdmb.rdmb_povray_q)(rdmb_fnbase,
                                                 time_point=2000,
                                                 width=240,
                                                 height=240,
                                                 angle=5)
                     for rdmb_fnbase in rdmb_fnbases])

In [ ]:
display(Image.open(pngfnames_q[425]))

### Quantification

In [ ]:
imgs = [Image.open(pngfname_q) for pngfname_q in pngfnames_q]
imgs_crop_scale = [quant.crop_center(img, 128, 128)
                   .convert('L')
                   .resize((420, 420), resample=Image.LANCZOS)
                   for img in imgs]
imgs_mono = [quant.binarize(np.asarray(img))
             for img in imgs_crop_scale]

Ls = np.array([quant.quant(img)[0] for img in imgs_mono])
PCSs = np.array([quant.quant(img)[1] for img in imgs_mono])

df_uni = pd.DataFrame(np.c_[rdmb_fnbases,
                            pngfnames_q,
                            AAs.flatten(),
                            CCs.flatten(),
                            Ls,
                            PCSs],
                      columns=["file_base",
                               "png_file_q",
                               "A",
                               "C",
                               "lightness",
                               "complexity"])
df_uni.to_csv("uni.csv")


In [ ]:
df_uni[100:105]

### Rendering for visualization

In [ ]:
pngfnames_col = Parallel(n_jobs=36, verbose=1, max_nbytes=None)(
                    [delayed(rdmb_povray_color)(rdmb_fnbase,
                                                time_point=2000,
                                                width=200,
                                                height=200,
                                                rotx=0, roty=0, rotz=0,
                                                angle=5,
                                                mode="AC")
                     for rdmb_fnbase in rdmb_fnbases])

In [ ]:
display(Image.open(pngfnames_col[425]))

In [ ]:
imax=30
jmax=30

img = Image.new('RGBA', (180*jmax+20, 180*imax+20), color="white")

for i in range(imax):
    for j in range(jmax):
        im = Image.open(pngfnames_col[i*jmax+j])
        img.paste(im, (180*i, 180*(jmax-j-1)), mask=im)
        
img.convert(mode="RGB").save("uni_grid_color.jpg", quality=95)

display(img.resize((800, 800), Image.LANCZOS))